# Data Proprocessing

In [60]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import os, sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))
# Set max rows and columns to display
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [61]:
# Read the dataset
df = pd.read_csv('../data/cleaned_data.csv', low_memory=False, index_col=False)

In [62]:
# Explore the first five rows
df.head(5)

,Unnamed: 0,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,NaN,Yes,No,119300,More than 6 months,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,NaN,Yes,No,119300,More than 6 months,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,NaN,Yes,No,119300,More than 6 months,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,NaN,Yes,No,119300,More than 6 months,119300.00,Monthly,220.1628,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,54.824561,0.0
4,4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,NaN,Yes,No,119300,More than 6 months,119300.00,Monthly,220.1628,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [63]:
# Convert 'CapitalOutstanding' to numeric, coerce errors to handle non-numeric values (e.g., empty strings or NaNs)
df['CapitalOutstanding'] = pd.to_numeric(df['CapitalOutstanding'], errors='coerce')

# Fill missing values if necessary, e.g., with 0 or the column's mean
df['CapitalOutstanding'] = df['CapitalOutstanding'].fillna(0)  # Or df['CapitalOutstanding'].fillna(df['CapitalOutstanding'].mean())

# Ensure it's a float type
df['CapitalOutstanding'] = df['CapitalOutstanding'].astype(float)

# Verify the conversion
print(df['CapitalOutstanding'].dtype)

float64


In [64]:
# Get categorical columns
categorical_columns = df.select_dtypes(include='object').columns
# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Column': categorical_columns,
    'DataType': [df[col].dtype for col in categorical_columns],
    'NumUniqueValues': [df[col].nunique() for col in categorical_columns]
})

# Print the summary DataFrame
summary_df

,Column,DataType,NumUniqueValues
0,TransactionMonth,object,23
1,Citizenship,object,4
2,LegalType,object,6
3,Title,object,5
4,Language,object,1
5,Bank,object,11
6,AccountType,object,3
7,MaritalStatus,object,3
8,Gender,object,3
9,Country,object,1


Key Insights from the Summary:

Columns like Language, Country, ItemType, StatutoryClass, and StatutoryRiskType don't provide variability.

Drop these columns from the dataset.

Model (411 unique values) could add complexity.

Consider reducing cardinality by grouping or applying target encoding.

Columns like AlarmImmobiliser, TrackingDevice, and NewVehicle have only 2 unique values.

TransactionMonth should be converted to a date format.

Extract useful features like year, month, and quarter if necessary.

CapitalOutstanding should be numeric and can be used as a continuous feature.

In [65]:
columns_to_drop = ['Language', 'Country', 'ItemType', 'StatutoryClass', 'StatutoryRiskType']
df = df.drop(columns=columns_to_drop)

Convert the CapitalOutstanding to numeric format

In [66]:
df.columns

Index(['Unnamed: 0', 'UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Bank',
       'AccountType', 'MaritalStatus', 'Gender', 'Province', 'PostalCode',
       'MainCrestaZone', 'SubCrestaZone', 'mmcode', 'VehicleType',
       'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity',
       'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate',
       'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding',
       'NewVehicle', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'TotalPremium', 'TotalClaims'],
      dtype='object')

In [67]:
# Convert boolean 'IsVATRegistered' to integer
df['IsVATRegistered'] = df['IsVATRegistered'].astype(int)

### Convert the TransactionMonth to a datetime format and extract relevant features.

In [68]:
df.columns

Index(['Unnamed: 0', 'UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Bank',
       'AccountType', 'MaritalStatus', 'Gender', 'Province', 'PostalCode',
       'MainCrestaZone', 'SubCrestaZone', 'mmcode', 'VehicleType',
       'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity',
       'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate',
       'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding',
       'NewVehicle', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'TotalPremium', 'TotalClaims'],
      dtype='object')

In [72]:
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])
df['TransactionYear'] = df['TransactionMonth'].dt.year
df['TransactionMonthOnly'] = df['TransactionMonth'].dt.month
df['TransactionQuarter'] = df['TransactionMonth'].dt.quarter
# Drop the original datetime column if it's no longer needed
df = df.drop(columns=['TransactionMonth'])

Check the Distribution of Title, MaritalStatus, and Gender

In [73]:
print(f'Gender Distribution:\n{df.Gender.value_counts()}')
print(f'Title Distribution:\n {df.Title.value_counts()}')
print(f'Marital Status Distribution:\n {df.MaritalStatus.value_counts()}')

Gender Distribution:
Gender
Not specified    940990
Male              42817
Female             6755
Name: count, dtype: int64
Title Distribution:
 Title
Mr      933555
Mrs      45850
Ms       13269
Miss      6614
Dr         810
Name: count, dtype: int64
Marital Status Distribution:
 MaritalStatus
Not specified    986208
Single             4254
Married            1377
Name: count, dtype: int64


In [74]:
print(f'Gender Distribution:\n{df.Gender.value_counts()}')
print(f'Title Distribution:\n {df.Title.value_counts()}')
print(f'Marital Status Distribution:\n {df.MaritalStatus.value_counts()}')

Gender Distribution:
Gender
Not specified    940990
Male              42817
Female             6755
Name: count, dtype: int64
Title Distribution:
 Title
Mr      933555
Mrs      45850
Ms       13269
Miss      6614
Dr         810
Name: count, dtype: int64
Marital Status Distribution:
 MaritalStatus
Not specified    986208
Single             4254
Married            1377
Name: count, dtype: int64


High Proportion of Unspecified Gender:

A significant portion of the Gender values (950,526) is missing or unspecified.

These unspecified gender values can be filled using the information from the Title column where available.
Title-to-Gender Mapping:

Titles like Mr, Mrs, Miss, and Ms provide a clear indication of gender.
Dr is ambiguous and may not specify gender directly.
Approach to Impute Missing Genders Using Titles:

Mr → Male

Mrs, Miss, Ms → Female

Dr → Leave as Not specified (since it’s ambiguous)

Balanced Gender Distribution:

The number of male and female entries is now significantly more balanced compared to the initial state where a large portion of the Gender values were unspecified.

Dropping the MaritalStatus column is a reasonable decision given the high proportion of unspecified values

In [76]:
# Drop the 'MaritalStatus' column from the DataFrame
df = df.drop(columns=['MaritalStatus'])

In [77]:
# Drop the specified columns from the DataFrame
df = df.drop(columns=['Unnamed: 0', 'UnderwrittenCoverID', 'PolicyID'])

Encoding Strategy:

Label Encoding: For binary or low-cardinality features.
One-Hot Encoding: For moderate cardinality features where the number of new columns remains manageable.
Target Encoding: For high cardinality features to reduce the number of new features while capturing useful patterns.
Frequency Encoding: Another option for high cardinality features, where you replace categories with their frequency in the dataset.

In [78]:
# Define cardinality thresholds
low_cardinality_threshold = 10
high_cardinality_threshold = 20

# Identify categorical columns
categorical_cols = df.select_dtypes(include='object').columns

# Separate features by cardinality
low_cardinality_features = [col for col in categorical_cols if df[col].nunique() <= low_cardinality_threshold]
moderate_cardinality_features = [col for col in categorical_cols if low_cardinality_threshold < df[col].nunique() <= high_cardinality_threshold]
high_cardinality_features = [col for col in categorical_cols if df[col].nunique() > high_cardinality_threshold]

# Label Encoding for low cardinality features
label_encoders = {}
for feature in low_cardinality_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# One-Hot Encoding for moderate cardinality features
one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)
for feature in moderate_cardinality_features:
    encoded = one_hot_encoder.fit_transform(df[[feature]])
    encoded_df = pd.DataFrame(encoded, columns=one_hot_encoder.get_feature_names_out([feature]))
    df = pd.concat([df, encoded_df], axis=1)
    df = df.drop(columns=[feature])

# Frequency Encoding for high cardinality features (Example)
for feature in high_cardinality_features:
    freq = df[feature].value_counts()
    df[feature + '_freq'] = df[feature].map(freq)
    df = df.drop(columns=[feature])



# Save or proceed with further analysis/modeling
# df.to_csv('preprocessed_data.csv', index=False)  # Uncomment to save the preprocessed data

In [79]:
# Verify the transformations
df.head()

,IsVATRegistered,Citizenship,LegalType,Title,AccountType,Gender,Province,PostalCode,mmcode,VehicleType,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,VehicleIntroDate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,SumInsured,TermFrequency,CalculatedPremiumPerTerm,Section,Product,TotalPremium,TotalClaims,TransactionYear,TransactionMonthOnly,TransactionQuarter,Bank_Capitec Bank,Bank_First National Bank,Bank_FirstRand Bank,Bank_Investec Bank,Bank_Ithala Bank,Bank_Mercantile Lisbon Bank,Bank_Nedbank,Bank_Old Mutual,Bank_RMB Private Bank,Bank_Standard Bank,MainCrestaZone_Cape Province (Cape Town),MainCrestaZone_Cape Province (East and North of Cape Town),"MainCrestaZone_Ciskei, Cape Mid 1",MainCrestaZone_East London,MainCrestaZone_Johannesburg,MainCrestaZone_Karoo 1 (Northeast of Cape Town),"MainCrestaZone_Langkloof, Coast 2, Coast 1",MainCrestaZone_Natal,MainCrestaZone_Natal (Durban),MainCrestaZone_Oranje Free State,MainCrestaZone_Port Elizabeth,MainCrestaZone_Rand East,"MainCrestaZone_Tembu 2, Cape Mid 2, Cape Mid West, Tembu 1",MainCrestaZone_Transvaal (Pretoria),MainCrestaZone_Transvaal (all except Pretoria),bodytype_C/C,bodytype_C/P,bodytype_CCL,bodytype_D/C,bodytype_D/S,bodytype_H/B,bodytype_MPV,bodytype_P/V,bodytype_S/C,bodytype_S/D,bodytype_S/W,bodytype_SUV,bodytype_nan,ExcessSelected_Mobility - Metered Taxis - R2000,ExcessSelected_Mobility - Metered Taxis - R5000,ExcessSelected_Mobility - R250,ExcessSelected_Mobility - Taxi with value less than R100 000 - R3 000,ExcessSelected_Mobility - Taxi with value more than R100 000 - R10 000,ExcessSelected_Mobility - Taxi with value more than R100 000 - R5 000,ExcessSelected_Mobility - Taxi with value more than R100 000 - R5 000 (April),ExcessSelected_Mobility - Taxi with value more than R100 000 - R7 500,ExcessSelected_Mobility - Trailers,ExcessSelected_Mobility - Windscreen,ExcessSelected_Mobility - Windscreen (Feb2015),ExcessSelected_No excess,CoverGroup_Asset Value Preserver,CoverGroup_Baggage/Luggage,CoverGroup_Basic Excess Waiver,CoverGroup_Cash Takings,CoverGroup_Comprehensive - Taxi,CoverGroup_Credit Protection,CoverGroup_Deposit Cover,"CoverGroup_Fire,Theft and Third Party",CoverGroup_Income Protector,CoverGroup_Motor Comprehensive,CoverGroup_Standalone passenger liability,CoverGroup_Third Party Only,CoverGroup_Trailer,SubCrestaZone_freq,make_freq,Model_freq,CoverCategory_freq,CoverType_freq
0,1,0,0,2,0,1,2,1459,44069150.0,4,2004,6.0,2597.0,130.0,4.0,NaN,1,0,119300.0,1,0.01,1,25.0000,1,2,21.929825,0.0,2015,3,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42168,41940.0,180.0,97313,103763
1,1,0,0,2,0,1,2,1459,44069150.0,4,2004,6.0,2597.0,130.0,4.0,NaN,1,0,119300.0,1,0.01,1,25.0000,1,2,21.929825,0.0,2015,5,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42168,41940.0,180.0,97313,103763
2,1,0,0,2,0,1,2,1459,44069150.0,4,2004,6.0,2597.0,130.0,4.0,NaN,1,0,119300.0,1,0.01,1,25.0000,1,2,0.000000,0.0,2015,7,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42168,41940.0,180.0,97313,103763
3,1,0,0,2,0,1,2,1459,44069150.0,4,2004,6.0,2597.0,130.0,4.0,NaN,1,0,119300.0,1,119300.00,1,220.1628,1,2,54.824561,0.0,2015,5,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,